# Download and import of packages

In [ ]:
!python -m pip install pyyaml
import sys, os, distutils.core

In [ ]:
# Detectron 2
!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'

In [ ]:
import torch, detectron2
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

In [ ]:
# Andre pakker

# Some basic setup:
# Setup detectron2 logger
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
from google.colab.patches import cv2_imshow
import pandas as pd

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.structures import BoxMode

In [ ]:
import os
import cv2
from detectron2.data import DatasetCatalog, MetadataCatalog
import os
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg
from detectron2 import model_zoo
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
from detectron2.data.datasets import convert_to_coco_json
import shutil
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg
from detectron2 import model_zoo

# Load in the images from GitHub

In [ ]:
from getpass import getpass
import subprocess

# Get GitHub token
token = getpass("Enter your GitHub token: ")

# Define repository URL
repo_url = f"https://{token}@github.com/casperbak1/Dataprojekt.git"

# Kloner main branch, og kun seneste commit (depth 1)
subprocess.run(["git", "clone", "--branch", "main", "--depth", "1", repo_url])

repo_path = "Dataprojekt"
if os.path.exists(repo_path):  # Hvis der findes en GitHub sti "Dataprojekt"
    subprocess.run(["git", "sparse-checkout", "init", "--cone"], cwd=repo_path) # Så klon mappen "Data/Clean Data/Overbite Data"
    subprocess.run(["git", "sparse-checkout", "set", "Data/Clean Data/Overbite Data"], cwd=repo_path)

print("Repository cloned with only the 'Overbite Data' folder.")

# Initialise the data for the model

# Training images

In [ ]:
# Load the annotations from the CSV file
ANNOTATIONS_FILE = "Dataprojekt/Data/Clean Data/Overbite Data/Updated_Labels.csv"
annotations_df = pd.read_csv(ANNOTATIONS_FILE)

# Sti til dataen
DATASET_PATH = "Dataprojekt/Data/Clean Data/Overbite Data/Annotated Data"

def my_dataset_function():
    dataset_dicts = []

    # Group annotations by filename (in case multiple keypoints exist for an image)
    grouped_annotations = annotations_df.groupby("Filename")

    for idx, filename in enumerate(os.listdir(DATASET_PATH)):
        if filename.endswith((".jpg", ".png", ".jpeg")):
            record = {}
            file_path = os.path.join(DATASET_PATH, filename)

            # Read image dimensions
            height, width = cv2.imread(file_path).shape[:2]

            # Initialize the dataset record
            record["file_name"] = file_path
            record["image_id"] = idx
            record["height"] = height
            record["width"] = width

            # Default empty annotation list
            record["annotations"] = []

            # Check if the file has keypoint annotations
            if filename in grouped_annotations.groups:
                keypoints_list = []
                for _, row in grouped_annotations.get_group(filename).iterrows():
                    x, y = row["X"], row["Y"]
                    keypoints_list.append(x)  # X-coordinate
                    keypoints_list.append(y)  # Y-coordinate
                    keypoints_list.append(2)  # Visibility flag (0=not visible, 1=occluded, 2=visible)

                # Create the annotation entry
                annotation = {
                    "bbox": [0, 0, width, height],  # Dummy bbox covering entire image
                    "bbox_mode": BoxMode.XYWH_ABS,
                    "category_id": 0,  # If you have multiple classes (Vi har 1)
                    "keypoints": keypoints_list,
                    "num_keypoints": len(keypoints_list) // 3
                }
                record["annotations"].append(annotation)

            dataset_dicts.append(record)

    return dataset_dicts

# Register the dataset
DatasetCatalog.register("Overbite_Data", my_dataset_function)
MetadataCatalog.get("Overbite_Data").set(
    thing_classes=["object"],  # Modify for actual class names (Mangler godt navn)
    keypoint_names=["keypoint"],  # Name of keypoints (Mangler endnu bedre navn)
    keypoint_flip_map=[]  # Add keypoint flip pairs if needed (Nope)
)

# Test if it works
dataset_dicts = DatasetCatalog.get("Overbite_Data")
print(f"Loaded {len(dataset_dicts)} images with keypoints.")

# Training validation images

In [ ]:
# Funktion til at hente verifikationsdata
def my_validation_function():
    dataset_dicts = []
    DATASET_PATH = "Dataprojekt/Data/Clean Data/Overbite Data/Annotated Verification Data"

    grouped_annotations = annotations_df.groupby("Filename")

    for idx, filename in enumerate(os.listdir(DATASET_PATH)):
        if filename.endswith((".jpg", ".png", ".jpeg")):
            record = {}
            file_path = os.path.join(DATASET_PATH, filename)
            height, width = cv2.imread(file_path).shape[:2]

            record["file_name"] = file_path
            record["image_id"] = idx
            record["height"] = height
            record["width"] = width
            record["annotations"] = []

            if filename in grouped_annotations.groups:
                keypoints_list = []
                for _, row in grouped_annotations.get_group(filename).iterrows():
                    x, y = row["X"], row["Y"]
                    keypoints_list.append(x)
                    keypoints_list.append(y)
                    keypoints_list.append(2)  # Visibility flag

                annotation = {
                    "bbox": [0, 0, width, height],  # Dummy bbox
                    "bbox_mode": BoxMode.XYWH_ABS,
                    "category_id": 0,
                    "keypoints": keypoints_list,
                    "num_keypoints": len(keypoints_list) // 3
                }
                record["annotations"].append(annotation)

            dataset_dicts.append(record)

    return dataset_dicts

# Registrér valideringsdatasæt
DatasetCatalog.register("Overbite_Validation", my_validation_function)
MetadataCatalog.get("Overbite_Validation").set(
    thing_classes=["object"],
    keypoint_names=["keypoint"],
    keypoint_flip_map=[]
)

# Train the model

In [ ]:
import os
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg
from detectron2 import model_zoo
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
from detectron2.data.datasets import convert_to_coco_json

# Create the configuration
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Keypoints/keypoint_rcnn_R_101_FPN_3x.yaml"))  # Load Keypoint-RCNN config
cfg.DATASETS.TRAIN = ("Overbite_Data",)  # Training dataset
cfg.DATASETS.TEST = ("Overbite_Validation",)  # Validation dataset
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Keypoints/keypoint_rcnn_R_101_FPN_3x.yaml")  # Pretrained weights
cfg.SOLVER.IMS_PER_BATCH = 2  # Batch size (Antal billeder per batch) (Større skulle gerne give god træning, men vi har begrænset GPU i "Colab")
cfg.SOLVER.BASE_LR = 0.00025  # Learning rate
cfg.SOLVER.MAX_ITER = 8000  # Antal træningsiterationer
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128  # Antal "Regions Of Interest" per billede
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1  # Antal klasser
cfg.MODEL.KEYPOINT_ON = True  # Enable keypoint detection
cfg.MODEL.ROI_KEYPOINT_HEAD.NUM_KEYPOINTS = 1  # Antal keypoints


cfg.TEST.KEYPOINT_OKS_SIGMAS = [0.1] # Set OKS sigma for 1 keypoint (Default is 17 for COCO)

# Automatisk validering hver 500. iteration
cfg.TEST.EVAL_PERIOD = 500

# Output directory for modellen
cfg.OUTPUT_DIR = "./output/Overbite_Model"
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)

# Konverter valideringsettet til COCO JSON format
coco_annotation_path = os.path.join(cfg.OUTPUT_DIR, "Overbite_Validation_coco_format.json")
convert_to_coco_json("Overbite_Validation", coco_annotation_path)
print(f"Validation set converted to COCO format: {coco_annotation_path}")

# Inkluder evaluatoren
class TrainerWithEval(DefaultTrainer):
    @classmethod
    def build_evaluator(cls, cfg, dataset_name):
        return COCOEvaluator(dataset_name, cfg, False, output_dir=cfg.OUTPUT_DIR)

# Træn modellen med evaluering
trainer = TrainerWithEval(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

# Kør en sidste evaluering på valideringsdataen
val_loader = build_detection_test_loader(cfg, "Overbite_Validation")
inference_on_dataset(trainer.model, val_loader, COCOEvaluator("Overbite_Validation", cfg, False, output_dir=cfg.OUTPUT_DIR))

print("Training and validation completed!")